In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import date,time,datetime,timedelta

In [2]:
PATH=r'D:\WORKS\PolimerSolve\Comsol\tables\1D'

In [9]:
def df_from_comsol(
    name,
    from_name=r'\out.csv',
    temp_name=r'\temp.csv',
):
    with open(PATH + from_name, 'r') as file:
        lines = file.readlines()

    with open(PATH + temp_name, 'w') as file:
        for line in lines[4:]:
            file.write(line)
    df = pd.read_csv(PATH + r'\temp.csv')
    df = df.rename(columns={
        '% k1': 'k1',
        '% k2': 'k2',
        df.columns[1]: 'target',
    })
    df['name'] = name
    df['time'] = datetime.now().replace(second=0, microsecond=0)
    return df


def add_notes(from_name=r'\out.csv'):
    name = ''
    while True:
        name = input('Name: ')
        if (name != '') and (name != 'q'):
            try:
                old_df = pd.read_csv(PATH + r'\base.csv',index_col=0)
                new_df = pd.concat(
                    [old_df, df_from_comsol(name, from_name=from_name)],
                    ignore_index=True,
                )
            except FileNotFoundError:
                old_df = None
                new_df = df_from_comsol(name, from_name=from_name)
            new_df.to_csv(PATH + r'\base.csv')

            return new_df, old_df
        elif name == 'q':
            break

# Add notes

In [40]:
base,old_df = add_notes()

# Check all notes

In [41]:
df = pd.read_csv(PATH+'/base.csv',index_col=0)
df['name'].unique()

array(['start', 'small', 'big'], dtype=object)

In [42]:
fig = go.Figure()
fig.update_layout(margin={'r': 0, 'l': 0, 't': 0, 'b': 0})
fig.add_traces(
    go.Scatter(
        x=df['k1'],
        y=df['target'],
        text=df['name'],
        mode='markers',
        hovertemplate='k1: %{x}<br>integral: %{y:.2E} <br> %{text}',
        name='',
    ))

# Check curent file

In [14]:
out_name=input('Write task name: ')
cur_filename ='\\'+out_name+'.csv'
df = pd.read_csv(PATH+r'\cleared' + cur_filename)
df = df.rename(columns={
    '% k1': 'k1',
    df.columns[1]: 'target',
})
fig = go.Figure()
fig.update_layout(margin={'r': 0, 'l': 0, 't': 0, 'b': 0})
fig.add_traces(
    go.Scatter(
        x=df['k1'],
        y=df['target'],
        hovertemplate='k1: %{x}<br>integral: %{y:.2f}',
        name='',
    ))